In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#load training dataset
ratings = pd.read_csv('train_dt/ratings_small.csv')

movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')

credits = pd.read_csv('train_dt/credits.csv')

keywords = pd.read_csv('train_dt/keywords.csv')

links = pd.read_csv('train_dt/links.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_8792\1884070253.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')


In [3]:
#load test dataset
test_dt = np.load('test_dt/Dataset.npy')
test_dt = pd.DataFrame(test_dt)

test_dt = test_dt[0].str.split(",", expand=True).rename(columns={0:'user_id', 1:"imdb_id", 2:"rating", 3:'rating_date'})
test_dt.head()

,user_id,imdb_id,rating,rating_date
0,ur4592644,tt0120884,10,16 January 2005
1,ur3174947,tt0118688,3,16 January 2005
2,ur3780035,tt0387887,8,16 January 2005
3,ur4592628,tt0346491,1,16 January 2005
4,ur3174947,tt0094721,8,16 January 2005


In [5]:
temp = pd.DataFrame(test_dt['user_id'].value_counts())
row_list = list(temp.index.values)
temp.reset_index(inplace = True)
temp.columns = ['user_id','rating_cnt']
temp = temp.drop(temp[temp['rating_cnt'] < 5].index)

In [16]:
unique_list = list(temp['user_id'])

In [ ]:
test_dt.head()


In [15]:
unique_list in 'ur4592644'

False

In [11]:
test_dt['user_id'].isin(unique_list)

0          False
1           True
2           True
3          False
4           True
           ...  
4669815     True
4669816     True
4669817    False
4669818     True
4669819     True
Name: user_id, Length: 4669820, dtype: bool

In [ ]:
#preprocessing
#movies metadata preprocessing
movies_metadata.dropna(inplace = True)

#'id'형식이 날짜 형식인 경우 -> 잘못된 경우
#ex) ValueError: Unable to parse string "2012-09-29" at position 29502
movies_metadata['isIdRight'] = movies_metadata['id'].str.contains('|'.join('-'))
movies_metadata = movies_metadata[movies_metadata['isIdRight'] == False]
movies_metadata.drop(['isIdRight'], axis = 1, inplace = True)

In [ ]:
def content_based_recommendation(keywords, credits, movies_metadata):
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')
    movies_metadata['id'] = movies_metadata['id'].astype('int')
    
    #content based dataset
    cb_trained_dt = movies_metadata
    cb_trained_dt = cb_trained_dt.merge(credits, on='id')
    cb_trained_dt = cb_trained_dt.merge(keywords, on='id')
    
    from ast import literal_eval
    #Parse the stringified features into their corresponding python objects
    features = ['cast', 'crew', 'keywords', 'genres']
    for feature in features:
        cb_trained_dt[feature] = cb_trained_dt[feature].apply(literal_eval)
        
    #apply func : 괄호 안의 함수를 dataframe 전체에 적용
    #related link : https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wideeyed&logNo=221559041280
    cb_trained_dt['director'] = cb_trained_dt['crew'].apply(get_director)

    features = ['cast', 'keywords', 'genres']
    for feature in features:
        cb_trained_dt[feature] = cb_trained_dt[feature].apply(get_list)

    features = ['cast', 'keywords', 'director', 'genres']
    for feature in features:
        cb_trained_dt[feature] = cb_trained_dt[feature].apply(clean_data)
        
    #soup라는 column에 keyword, cast, director, genres를 모두 묶어 cosine similarity를 측정할 수 있는 값들을 합쳐서 넣어둠
    cb_trained_dt['soup'] = cb_trained_dt.apply(create_soup, axis=1)
    
    return make_recommendation_user_input(cb_trained_dt)
#영화 관련자 중 배우만 따로 빼서(parsing해서) 보관하는 함수
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
#id 등으로 복잡하게 되어 있는 경우 이를 parsing해서 list로 return해주는 함수
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #if len(names) > 3: --> 3개 이상인 경우는 names에서 제거 : 없애고 실행함 (정확도 상승?)
            #names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []
#빈칸 제거 함수
def clean_data(x):
    #list 내부의 빈칸 제거 -> [cast, keywords, genres]
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x] #str.lower(대문자 -> 소문자 변경) + 빈칸 제거
    else:
        #string 내부의 빈칸 제거 -> [director]
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))#빈칸 제거
        else:
            return ''
#cosine similarity function을 위한 말뭉치 생성
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
#get input and return output
def get_searchTerms():
    searchTerms = [] 
    genres = input("What Movie Genre(if multiple, please separate them with a comma)? [Type 'skip' to skip] ")
    genres = " ".join(["".join(n.split()) for n in genres.lower().split(',')])
    if genres != 'skip':
        searchTerms.append(genres)

    actors = input("Who are some actors(within the genre)(if multiple, please separate them with a comma)? [Type 'skip' to skip] ")
    actors = " ".join(["".join(n.split()) for n in actors.lower().split(',')])
    if actors != 'skip':
        searchTerms.append(actors)

    directors = input("Who are some directors(within the genre)(if multiple, please separate them with a comma)? [Type 'skip' to skip] ")
    directors = " ".join(["".join(n.split()) for n in directors.lower().split(',')])
    if directors != 'skip':
        searchTerms.append(directors)

    keywords = input("What are some of the keywords(ex, friendship)? (if multiple, please separate them with a comma)? [Type 'skip' to skip] ")
    keywords = " ".join(["".join(n.split()) for n in keywords.lower().split(',')])
    if keywords != 'skip':
        searchTerms.append(keywords)

    return searchTerms
def make_recommendation_user_input(metadata=cb_trained_dt):
    new_row = metadata.iloc[-1,:].copy() #creating a copy of the last row of the 
  #dataset, which we will use to input the user's input
  
  #grabbing the new wordsoup from the user
    searchTerms = get_searchTerms()  
    new_row.iloc[-1] = " ".join(searchTerms) #adding the input to our new row
  
  #adding the new row to the dataset
    metadata = metadata.append(new_row)
  
  #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(metadata['soup'])

  #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix
  
  #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  #matching the similarities to the movie titles and ids
    ranked_titles = []
    for i in range(1, 5 + 1):
        indx = sim_scores[i][0] 
        ranked_titles.append([metadata['title'].iloc[indx], 'https://imdb.com/title/' + metadata['imdb_id'].iloc[indx]]) #대신 사이트 주소 출력
  
    return ranked_titles

In [ ]:
rank_list = content_based_recommendation(keywords, credits, movies_metadata)

for i in rank_list:
    print(i)